In [ ]:
import requests
import numpy as np
import os
import time
import joblib
from dotenv import load_dotenv
import json
import time
load_dotenv()

In [ ]:
ids = np.load('RandomSongsIds.npy')
print(len(ids))

In [223]:
model = joblib.load("../MSD_RFR_PredictiveModel.pkl")
scaler = joblib.load("../MSD_RFR_Scaler.pkl")

In [224]:
def decade_classify(year):
    if year >= 1922 and year <= 1929:
        return 1
    elif year >= 1930 and year <= 1939:
        return 2
    elif year >= 1940 and year <= 1949:
        return 3
    elif year >= 1950 and year <= 1959:
        return 4
    elif year >= 1960 and year <= 1969:
        return 5
    elif year >= 1970 and year <= 1979:
        return 6
    elif year >= 1980 and year <= 1989:
        return 7
    elif year >= 1990 and year <= 1999:
        return 8
    elif year >= 2000 and year <= 2011:
        return 9
    else:
        return -1

def tempo_transform(tempo):
    if tempo >= 200:
        return tempo / 2
    else:
        return tempo

In [225]:
ENV_FILE = '.env'

def read_env_file():
    with open(ENV_FILE, 'r') as file:
        return file.readlines()

def write_env_file(lines):
    with open(ENV_FILE, 'w') as file:
        file.writelines(lines)

def update_env_variable(key, value):
    lines = read_env_file()
    updated = False

    for i, line in enumerate(lines):
        if line.startswith(f"{key}="):
            lines[i] = f"{key}={value}\n"
            updated = True
            break

    if not updated:
        lines.append(f"{key}={value}\n")

    write_env_file(lines)

def get_access_token():
    data = {
        'grant_type': 'client_credentials',
        'client_id': os.getenv('CLIENT_ID'),
        'client_secret': os.getenv('CLIENT_SECRET')
    }

    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    try:
        response = requests.post('https://accounts.spotify.com/api/token', data=data, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as err:
        print(f"Error: {err}")
        return None

def check_token():
    token = os.getenv("AUTH_TOKEN")
    token_time = os.getenv("AUTH_TIME")

    if not token or not token_time or int(time.time()) > 1727140944:
        token_data = get_access_token()
        if token_data:
            update_env_variable("AUTH_TOKEN", token_data['access_token'])
            update_env_variable("AUTH_TIME", str(int(time.time()) + token_data['expires_in']))
            return token_data['access_token']
        else:
            return None
    else:
        return token

In [ ]:
compare = []

for id in ids:
    basic = requests.get(f"https://api.spotify.com/v1/tracks/{id}", headers={
        "Authorization": f"Bearer {check_token()}"
    })
    basic_json = basic.json()

    time.sleep(1.5)

    features = requests.get(f"https://api.spotify.com/v1/audio-features/{id}", headers={
        "Authorization": f"Bearer {check_token()}"
    })
    feat_json = features.json()

    body = {
        "year": int(basic_json["album"]["release_date"].split("-")[0]),
        "duration": feat_json["duration_ms"],
        "key": feat_json["key"],
        "loudness": feat_json["loudness"],
        "mode": feat_json["mode"],
        "tempo": feat_json["mode"],
        "signature": feat_json["time_signature"]
    }
    df = pd.DataFrame([body])
    df["year"] = df["year"].apply(decade_classify)
    df["tempo"] = df["tempo"].apply(tempo_transform)
    df_scaled = scaler.transform(df)

    prediction = model.predict(df_scaled)
    comparision = {
            "id": id,
            "real": basic_json["popularity"] / 100,
            "prediction": prediction.tolist()[0]
        }

    if comparision["real"] == 0:
        pass
    else:
        compare.append(comparision)
        print(len(compare))
        print(comparision)

In [196]:
with open("RandomSongsData.json", "w") as json_file:
    json.dump(compare, json_file, indent=4)